# Demo of loading CMIP data from cloud

Adapted from [Ryan Abernathey's demo](https://nbviewer.org/github/pangeo-data/pangeo-cmip6-examples/blob/master/basic_search_and_load.ipynb?source=post_page-----96b177abe396---------------------------------------).

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import time

In [ ]:
df = pd.read_csv(
    "https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv"
)
df_ta = df.query(
    "activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'tas' & experiment_id == 'historical' & member_id == 'r1i1p1f1'"
)

In [ ]:
df_subset = df.query(
    "activity_id=='CMIP' & table_id == 'Omon' & variable_id == 'tos' & experiment_id == '1pctCO2' & member_id == 'r1i1p1f1'"
)

## Load information about data

In [ ]:
## Open CSV file with information about available data
t0 = time.time()
df = pd.read_csv(
    "https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv"
)
t1 = time.time()
print("Done loading model information.")
print(f"Elapsed time: {(t1-t0)/60:.1f} seconds")

## subset for the data we want
df_subset = df.query(
    "activity_id=='CMIP' & table_id == 'Omon' & variable_id == 'tos' & experiment_id == '1pctCO2' & member_id == 'r1i1p1f1'"
)

## list of models
models = [
    ("NCAR", "CESM2"),
    ("NCAR", "CESM2-WACCM"),
    ("CSIRO", "ACCESS-ESM1-5"),
    ("MIROC", "MIROC6"),
    ("MPI-M", "MPI-ESM1-2-LR"),
    ("IPSL", "IPSL-CM6A-LR"),
    ("NASA-GISS", "GISS-E2-2-G"),
]
models_ = [m[1] for m in models]

## subset for models
df_models = df_subset[df_subset["source_id"].isin(models_)]

## get models on native grid
df_models = df_models[df_models["grid_label"] == "gn"]

In [ ]:
{name: link for name, link in zip(df_models["source_id"], df_models["zstore"])}

## Open the data

In [ ]:
# this only needs to be created once
gcs = gcsfs.GCSFileSystem(token="anon")

# get the path to a specific zarr store (the first one from the dataframe above)
zstore = df_models.zstore.values[-1]

# create a mutable-mapping-style interface to the store
mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated=True)